In [ ]:
import sklearn
from gensim.models import Word2Vec
import gensim

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io

In [ ]:
# file path
# find the dataset located online at https://osf.io/efrqt/
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/SigmaLaw-ABSA.csv')

In [ ]:
df.head()

,Sentence,Party,Sentiment,Overall Sentiment
0,Petitioner Jae Lee moved to the United States ...,"[[Petitioner Jae Lee,his,he],[]]","[[0,0,0],[]]",0
1,"In the 35 years he has spent in this country, ...","[[he_1,he_2,he_3],[]]","[[0,0,0],[]]",0
2,"In the 35 years he has spent in this country, ...","[[he_1,he_2],[]]","[[0,0],[]]",0
3,In the 35 years he has spent in this country,"[[he],[]]","[[0],[]]",0
4,"In 2008, federal officials received a tip from...","[[lee],[federal officials]]","[[-1],[1]]",-1


In [ ]:
!pip install stop-words
from stop_words import get_stop_words
stopwords = get_stop_words('en')
from textblob import Word
import re
import nltk
nltk.download('wordnet')

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=7c06e5def0c0667811cae7a90e6ae8214978d301ed2a91aefec96dc6175fe569
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Preprocessing function used to:


*   Remove stop words
*   Remove punctuation
*   Split up sentence
*   Clean data



In [ ]:
def utils_preprocess_text(txt, lst_regex=None, punkt=True, lower=True, lst_stopwords=None, stemm=False, lemm=True):
    ## regex (in case, before processing)
    if lst_regex is not None: 
        for regex in lst_regex:
            txt = re.sub(regex, '', txt)
    
    ## clean 
    ### separate sentences with '. '
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
    ### remove punctuations and characters
    txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
    ### strip
    txt = " ".join([word.strip() for word in txt.split()])
    ### lowercase
    txt = txt.lower() if lower is True else txt
            
    ## Tokenize (convert from string to list)
    lst_txt = txt.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]
                
    ## Lemmatization (convert the word into root word)
    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]

    ## remove leftover Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]
            
    ## back to string from list
    txt = " ".join(lst_txt)
    return txt

In [ ]:
df['clean_text'] = df['Sentence'].apply( lambda x:utils_preprocess_text(x, stemm = False, lemm = True, lst_stopwords=stopwords))
df['clean_text'].head()

0    petitioner jae lee moved united state south ko...
1    35 year spent country never returned south kor...
2     35 year spent country never returned south korea
3                                35 year spent country
4    2008 federal official received tip confidentia...
Name: clean_text, dtype: object

# Word embedding and model training
Now that the text has been preprocessed, we will split the sentences up and change them into a vectorized format using word embedding.

Then we will make the ML models

In [ ]:
import gensim.downloader as api
nlp = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# functions borrowed from Huyen's github
# BOW based approaches
nlp.init_sims(replace=True) # calling for using syn0norm

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
# functions borrowed from Huyen's github
# Tokenize, and apply word vector averaging to tokenized text
nltk.download('punkt')
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.model_selection import train_test_split
import logging
X = df['clean_text']
Y = df['Overall Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .2)

In [ ]:
X_train_tokenized = X_train.apply(lambda x: w2v_tokenize_text(x)).values
X_test_tokenized = X_test.apply(lambda x: w2v_tokenize_text(x)).values

X_train_word_average = word_averaging_list(nlp,X_train_tokenized)
X_test_word_average = word_averaging_list(nlp,X_test_tokenized)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  if sys.path[0] == '':


In [ ]:
nlp["comics"]

array([ 0.05675178,  0.0182522 , -0.05054455,  0.10108911, -0.03310521,
        0.00166265, -0.02187308,  0.05350038,  0.00591164, -0.03694777,
        0.14010596,  0.01352288,  0.08394534,  0.01256224, -0.05290921,
       -0.02216866,  0.05822969,  0.07685137, -0.00713092, -0.06295901,
        0.00182891, -0.03354858,  0.07980719, -0.0614811 , -0.09222164,
        0.00301124,  0.04817989,  0.10404493,  0.00245703, -0.12296219,
       -0.06887065, -0.07980719, -0.04049476,  0.10108911,  0.033253  ,
        0.06236784,  0.08276301,  0.02246425,  0.02911485,  0.14719993,
       -0.00199518, -0.00208755,  0.09576863,  0.05379596, -0.05054455,
        0.02468111, -0.00757429, -0.08571883,  0.01396626, -0.0399036 ,
       -0.10227144, -0.06325459, -0.06916623,  0.09517746, -0.05231805,
        0.01906505,  0.09517746,  0.04345058, -0.09695096, -0.08394534,
        0.07744253, -0.04374616, -0.033253  , -0.04492849, -0.06000318,
       -0.06236784, -0.14956458, -0.00702008, -0.01758714,  0.06

In [ ]:
len(X_test)

400

# Models


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree             # tree.DecisionTreeClassifier()
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm #clf = svm.SVC(decision_function_shape='ovo')
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_validate

**Random Forest**

In [ ]:
##### RANDOM FOREST ######
rf = RandomForestClassifier(random_state=42)

# Validate the model's performance using k-fold cross validation

cv_rf = cross_validate (rf, X_train_word_average, y_train, cv = 10)
print("Random Forest cross validation accuracy mean score: \n", cv_rf['test_score'].mean())

rf.fit(X_train_word_average, y_train)
y_pred_rf = rf.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred_rf, y_test))
print(classification_report(y_test, y_pred_rf))

Random Forest cross validation accuracy mean score: 
 0.615625
accuracy 0.6175
              precision    recall  f1-score   support

          -1       0.60      0.84      0.70       203
           0       0.85      0.52      0.64        66
           1       0.55      0.32      0.41       131

    accuracy                           0.62       400
   macro avg       0.67      0.56      0.58       400
weighted avg       0.63      0.62      0.60       400



**Logistic Regression**

In [ ]:
# Fit the classifiers into the vectorized training set

##### LOGISTIC REGRESSION ######
logreg = LogisticRegression(solver='lbfgs', max_iter=100)

# evaluate the accuracy score with cv
cv_log = cross_validate (logreg, X_train_word_average, y_train, cv = 10)
print("Logistic Regression CV accuracy mean score: \n", cv_log['test_score'].mean())
# the accuracy mean score for SigmaLaw-ABSA maxes out around 75 iterations

logreg.fit(X_train_word_average, y_train)
y_pred_log = logreg.predict(X_test_word_average)
print('accuracy on test set %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

Logistic Regression CV accuracy mean score: 
 0.5893750000000001
accuracy on test set 0.5925
              precision    recall  f1-score   support

          -1       0.61      0.77      0.68       203
           0       0.65      0.45      0.54        66
           1       0.53      0.39      0.45       131

    accuracy                           0.59       400
   macro avg       0.59      0.54      0.55       400
weighted avg       0.59      0.59      0.58       400



**SVM**

In [ ]:
##### SVM ######
svm = svm.SVC(decision_function_shape='ovo', probability=True)

# Validate the model's performance using k-fold cross validation

cv_svm = cross_validate (svm, X_train_word_average, y_train, cv = 10)
print("SVM CV accuracy mean score: \n", cv_svm['test_score'].mean())

svm.fit(X_train_word_average, y_train)
y_pred_svm = svm.predict(X_test_word_average)
print('accuracy on test set %s' % accuracy_score(y_pred_svm, y_test))
print(classification_report(y_test, y_pred_svm))

SVM CV accuracy mean score: 
 0.6250000000000001
accuracy on test set 0.6325
              precision    recall  f1-score   support

          -1       0.63      0.81      0.71       203
           0       0.81      0.53      0.64        66
           1       0.56      0.41      0.48       131

    accuracy                           0.63       400
   macro avg       0.67      0.58      0.61       400
weighted avg       0.64      0.63      0.62       400



**Decision Tree (X-G Boost)**

In [ ]:
##### Decision Tree ######
tree = tree.DecisionTreeClassifier()

# Validate the model's performance using k-fold cross validation

cv_tree = cross_validate (tree, X_train_word_average, y_train, cv = 20)
print("Decion Tree CV accuracy mean score: \n", cv_tree['test_score'].mean())

tree.fit(X_train_word_average, y_train)
y_pred_tree = tree.predict(X_test_word_average)
print('accuracy on test set %s' % accuracy_score(y_pred_tree, y_test))
print(classification_report(y_test, y_pred_tree))

Decion Tree CV accuracy mean score: 
 0.50625
accuracy on test set 0.54
              precision    recall  f1-score   support

          -1       0.59      0.61      0.60       203
           0       0.49      0.59      0.54        66
           1       0.47      0.41      0.44       131

    accuracy                           0.54       400
   macro avg       0.52      0.54      0.53       400
weighted avg       0.54      0.54      0.54       400



**Gradient Boost**

In [ ]:
##### Gradient Boost ######
gb = GradientBoostingClassifier(random_state=42)

# Validate the model's performance using k-fold cross validation

cv_gb = cross_validate (gb, X_train_word_average, y_train, cv = 10)
print("Gradient Boost cross validation accuracy mean score: \n", cv_gb['test_score'].mean())

gb.fit(X_train_word_average, y_train)
y_pred_gb = gb.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred_gb, y_test))
print(classification_report(y_test, y_pred_gb))

Gradient Boost cross validation accuracy mean score: 
 0.6031249999999999
accuracy 0.605
              precision    recall  f1-score   support

          -1       0.61      0.75      0.67       203
           0       0.75      0.58      0.65        66
           1       0.52      0.39      0.45       131

    accuracy                           0.60       400
   macro avg       0.63      0.57      0.59       400
weighted avg       0.60      0.60      0.60       400

